In [ ]:
import pandas as pd
from datetime import datetime
import numpy as np
from kerastuner.tuners import RandomSearch
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
from tensorflow.keras import layers
# Load the TensorBoard notebook extension
%load_ext tensorboard
import tensorflow as tf
import datetime
from keras.callbacks import EarlyStopping

stopper = EarlyStopping(patience = 10)

# Clear any logs from previous runs
!rm -rf ./logs/ 

train_df = pd.read_csv('data/features.csv')

train_x = train_df.drop(columns=['precio'])
train_y = train_df.precio

train_x = train_x.to_numpy()
train_y = train_y.to_numpy()

n_cols = train_x.shape[0]

def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Dense(units=hp.Int('units',
                                        min_value=32,
                                        max_value=64,
                                        step=16), activation = 'relu', input_shape=(43,)))
    for i in range(hp.Int('num_layers',4, 10)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=64,
                                            max_value=256,
                                            step=32),
                               activation='relu'))
    model.add(layers.Dense(1))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='mean_squared_error',
        metrics=['mae'])
    return model

tuner = RandomSearch(
    build_model,
    objective='mae',
    max_trials=100,
    executions_per_trial=1,
    directory='models/definitive_deep_model',
    project_name='deep_learning_with_keras_tuner')

tuner.search_space_summary()

tuner.search(x=train_x,
             y=train_y,
             epochs=10,
             validation_split=0.5)

tuner.results_summary()

In [ ]:
models = tuner.get_best_models(num_models=3)

In [ ]:
best_model_1 = models[0]
best_model_2 = models[1]
best_model_3 = models[2]

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

earlyStopper = EarlyStopping(patience = 10)

In [ ]:
best_model_1.fit(train_x, train_y, validation_split = 0.4, epochs = 300, callbacks = [earlyStopper])

In [ ]:
best_model_2.fit(train_x, train_y, validation_split = 0.4, epochs = 300, callbacks = [earlyStopper])

In [ ]:
best_model_3.fit(train_x, train_y, validation_split = 0.4, epochs = 100, callbacks = [earlyStopper])

In [ ]:
test_df = pd.read_csv('data/testModified.csv')
test_id = test_df.id
test_df = test_df.drop(columns=['id'])
test_df = test_df.to_numpy()

best_model_1_prediction = best_model_1.predict(test_df)
res = pd.DataFrame(best_model_1_prediction, index=test_id, columns=['precio'])
res = res.rename({'precio':'target'}, axis=1)
res.to_csv("predictions/deep_model_1_prediction.csv", header = True)

best_model_2_prediction = best_model_2.predict(test_df)
res = pd.DataFrame(best_model_2_prediction, index=test_id, columns=['precio'])
res = res.rename({'precio':'target'}, axis=1)
res.to_csv("predictions/deep_model_2_prediction.csv", header = True)

best_model_3_prediction = best_model_3.predict(test_df)
res = pd.DataFrame(best_model_3_prediction, index=test_id, columns=['precio'])
res = res.rename({'precio':'target'}, axis=1)
res.to_csv("predictions/deep_model_3_prediction.csv", header = True)

In [ ]:
best_model_avg_prediction = (best_model_1_prediction + best_model_2_prediction + best_model_3_prediction) / 3 
res = pd.DataFrame(best_model_avg_prediction, index=test_id, columns=['precio'])
res = res.rename({'precio':'target'}, axis=1)
res.to_csv("predictions/deep_model_avg_prediction.csv", header = True)